In [2]:
import os
import difflib
import zipfile
import lxml.etree as etree
import random
import re
import sys
from datetime import datetime
import pandas as pd

import django
import platform

sys.path.append('/home/leey/Documents/Data/tmv/BasicBrowser/')
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "BasicBrowser.settings")
django.setup()

# import from appended path
import parliament.models as pm
#from parliament.tasks import do_search, run_tm
import cities.models as cmodels
from django.contrib.auth.models import User
import tmv_app.models as tm

#from parsing_utils import POI, dehyphenate_with_space, clean_text, find_person_in_db_aph
#from aph_model_builder import *

In [3]:
data_dir = '/home/leey/Documents/Data/AustralianPoliticians/data-raw/'